In [1]:
%%writefile experiments/001_baseline/run_baseline.py
import subprocess
import os
import shutil
import time

def run_command(cmd):
    print(f"Running: {cmd}")
    subprocess.run(cmd, shell=True, check=True)

def main():
    # Configuration
    n_iter = 5000 # Reduced for baseline speed
    r_iter = 16    # Reduced for baseline speed
    iterations = 2 # Number of full loops
    
    # Paths
    packer_exe = "./tree_packer_v21"
    bp_exe = "./bp"
    rot_script = "optimize_rotation.py"
    submission_file = "submission.csv"
    
    # 1. Initial Packing
    print("--- Step 1: Initial Packing ---")
    # tree_packer_v21 generates submission.csv automatically (or submission_v21.csv?)
    # Based on santa-claude.ipynb, it generates submission_v21.csv
    # Let's check the source code of tree_packer_v21.cpp to be sure about output filename.
    # But assuming it follows the notebook logic:
    run_command(f"{packer_exe} -n {n_iter} -r {r_iter}")
    
    # Check what file was created
    if os.path.exists("submission_v21.csv"):
        shutil.copy("submission_v21.csv", submission_file)
    elif not os.path.exists(submission_file):
        print("Error: No submission file generated by packer.")
        # If packer failed to generate, we might need to check if it outputs to stdout and we need to capture it?
        # The C++ code seemed to write to file.
        return

    for i in range(iterations):
        print(f"\n=== Iteration {i+1}/{iterations} ===")
        
        # 2. Backward Propagation
        print("--- Step 2: Backward Propagation ---")
        run_command(f"{bp_exe} {submission_file} {submission_file}")
        
        # 3. Rotation Optimization
        print("--- Step 3: Rotation Optimization ---")
        run_command(f"python {rot_script} {submission_file} {submission_file} 1")
        
        # Save intermediate result
        shutil.copy(submission_file, f"submission_iter_{i+1}.csv")
        
    print("\nBaseline run complete.")

if __name__ == "__main__":
    main()


Writing experiments/001_baseline/run_baseline.py
